In [ ]:
from lecture import *

# Introduction to programming in Python
### [Gerard Gorman](http://www.imperial.ac.uk/people/g.gorman)

# Lecture 4: Computing with NumPy arrays

Learning objectives: 

* Learn how to compute using [Numerical Python (*NumPy*)](http://www.numpy.org/).
* Know how to handle multidimensional arrays.

## Vectors and arrays

You have known **vectors** since high school mathematics, *e.g.*, point $(x,y)$ in the plane, point $(x,y,z)$ in space. In general, we can describe a vector $v$ as an $n$-tuple of numbers: $v=(v_0,\ldots,v_{n-1})$. One way to store vectors in Python is by using *lists*: $v_i$ is stored as *v[i]*.

**Arrays** are a generalization of vectors where we can have multiple indices: $A_{i,j}$, $A_{i,j,k}$. In Python code this is represented as a nested list (see previous lecture), accessed as *A[i][j]*, *A[i][j][k]*.

Example: table of numbers, one index for the row, one for the column
$$
\left\lbrack\begin{array}{cccc}
0 & 12 & -1 & 5q
-1 & -1 & -1 & 0\cr
11 & 5 & 5 & -2
\end{array}\right\rbrack
\hspace{1cm}
A =
\left\lbrack\begin{array}{ccc}
A_{0,0} & \cdots &  A_{0,n-1}\cr
\vdots & \ddots &  \vdots\cr
A_{m-1,0} & \cdots & A_{m-1,n-1}
\end{array}\right\rbrack
$$
The number of indices in an array is the *rank* or *number of dimensions*. Using these terms, a vector can be described as a one-dimensional array, or rank 1 array.

In practice, we use [Numerical Python (*NumPy*)](http://www.numpy.org/) arrays instead of lists to represent mathematical arrays because it is **much** faster for large arrays.

Let's consider an example where we store $(x,y)$ points along a curve in Python lists and numpy arrays:

In [ ]:
# Sample function
def f(x):
    return x**3

# Generate n points in [0,1]
n = 5
dx = 1.0/(n-1) # x spacing

xlist = [i*dx for i in range(n)] # Python lists
ylist = [f(x) for x in xlist]

# Turn these Python lists into Numerical Python (NumPy) arrays:
import numpy as np
x2 = np.array(xlist)
y2 = np.array(ylist)

Instead of first making lists with $x$ and $y = f (x)$ data, and then turning lists into arrays, we can make NumPy arrays
directly:

In [ ]:
n = 5                     # number of points
x2 = np.linspace(0, 1, n)    # generates n points between 0 and 1
y2 = np.zeros(n)             # n zeros (float data type by default)
for i in range(n):     
    y2[i] = f(x2[i])

List comprehensions create lists, not arrays, but we can do:

In [ ]:
y2 = np.array([f(xi) for xi in x2]) # list -> array

### When and where to use NumPy arrays

* Python lists can hold any sequence of any Python objects, however, NumPy arrays can only hold objects of the same type.
* Arrays are most efficient when the elements are of basic number types (*float*, *int*, *complex*).
* In that case, arrays are stored efficiently in the computer's memory and we can compute very efficiently with the array elements.
* Mathematical operations on whole arrays can be done without loops in Python. For example,

In [ ]:
import math

x = np.linspace(0, 2, 10001)
y = np.zeros(10001)
for i in range(len(x)):
    y[i] = math.sin(x[i])

can be coded as

In [ ]:
y = np.sin(x)

In the latter case the loop over all elements is now performed in a very efficient C function.

Operations on whole arrays, instead of using Python *for*-loops, is called vectorization and is a very **convenient**, **efficient** and therefore important programming technique to master.

Let's consider a simple vectorisation example: a loop to compute $x$ coordinates (*x2*) and $y=f(x)$ coordinates (*y2*) along a function curve:

In [ ]:
x2 = np.linspace(0, 1, n)
y2 = np.zeros(n)
for i in range(n):
    y2[i] = f(x2[i])

This computation can be replaced by:

In [ ]:
x2 = np.linspace(0, 1, n)
y2 = f(x2)

The advantage of this approach is:

* There is no need to allocate space for y2 (via the NumPy *zeros* function).
* There is no need for a loop.
* It is *much* faster.

## How vectorised functions work
Consider the function

In [ ]:
def f(x):
    return x**3

$f(x)$ is intended for a number $x$, *i.e.* a *scalar*. So what happens when we call *f(x2)*, where *x2* is an NumPy array? **The function simply evaluates $x^3$ for an array x**. NumPy supports arithmetic operations on arrays, which correspond to the equivalent operations on each element, *e.g.*:

In [ ]:
x**3                # x[i]**3 forr all i
np.cos(x)              # cos(x[i]) for all i
x**3 + x*np.cos(x)     # x[i]**3 + x[i]*cos(x[i]) for all i
x/3*np.exp(-x*0.5)     # x[i]/3*exp(-x[i]*0.5) for all i 

In each of these cases a highly optimised C function is actually called to evaluate the expression. In this example, the *cos* function called for an *array* is imported from *numpy* rather than from the *math* module which only acts on scalars.

Notes:

* Functions that can operate on arrays are called **vectorized functions**.
* Vectorization is the process of turning a non-vectorized expression/algorithm into a vectorized expression/algorithm.
* Mathematical functions in Python automatically work for both scalar and array (vector) arguments, *i.e.* no vectorization is needed by the programmer.


### Watch out for references Vs. copies of arrays!
Consider this code:

In [ ]:
a=x
a[-1] = 42
print(x[-1])

Notice what happened here - we changed a value in *a* but the corresponding value in *x* was also changed! This is because *a* refers to the same array as *x*. If you really want a seperate copy of *x* then we have to make an explicit copy:

In [ ]:
a = x.copy()

## Exercise 4.1: Fill lists and arrays with function values

A function with many applications in science is defined as:

$$h(x) = \frac{1}{\sqrt{2\pi}}\exp(-0.5x^2)$$

* Implement the above formula as a Python function. Call the function *h* and it should take just one argument, *x*.
* Create a NumPy array (call it $x$) that has 9 uniformly spaced points in [−4, 4].
* Create a second NumPy array (call it $y$) with the function $h(x)$.

In [ ]:
ok.grade('question-4_1')

## Generalised array indexing
We can select a slice of an array using *a[start:stop:inc]*, where the slice *start:stop:inc* implies a set of indices starting from *start*, up to *stop* in increments of *inc*. In fact, any integer list or array can be used to indicate a set of indices:

In [ ]:
a = np.linspace(1, 8, 8)
print(a)

In [ ]:
a[[1,6,7]] = 10 # i.e. set the elements with indicies 1,6, and 7 in the list to 10.
print(a)

In [ ]:
a[range(2,8,3)] = -2   # same as a[2:8:3] = -2
print(a)

Even boolean expressions can also be used to select part of an array(!)

In [ ]:
print(a[a < 0]) # pick out all negative elements

In [ ]:
a[a < 0] = a.max() # if a[i]<0, set a[i]=10
print(a)

## Exercise 4.2: Explore array slicing

* Create a NumPy array called *w* with 31 uniformly spaced values ranging from 0 to 3.
* Using array slicing, create a NumPy array called *wbits* that starts from the $4^{th}$ element of *w*, excludes the final element of *w* and selects every $3^{rd}$ element.

In [ ]:
ok.grade('question-4_2')

## 2D arrays
When we have a table of numbers,

$$
\left\lbrack\begin{array}{cccc}
0 & 12 & -1 & 5\cr
-1 & -1 & -1 & 0\cr
11 & 5 & 5 & -2
\end{array}\right\rbrack
$$

(*i.e.* a *matrix*) it is natural to use a two-dimensional array $A_{i,j}$ with one index for the rows and one for the columns:

$$
A = 
\left\lbrack\begin{array}{ccc}
A_{0,0} & \cdots &  A_{0,n-1}\cr
\vdots & \ddots &  \vdots\cr
A_{m-1,0} & \cdots & A_{m-1,n-1}
\end{array}\right\rbrack
$$

Let's recreate this array using NumPy:

In [ ]:
A = np.zeros((3,4))
A[0,0] = 0
A[1,0] = -1
A[2,0] = 11

A[0,1] = 12
A[1,1] = -1
A[2,1] = 5

A[0,2] = -1
A[1,2] = -1
A[2,2] = 5

# we can also use the same syntax that we used for nested lists

A[0][3] = 5
A[1][3] = 0
A[2][3] = -2

print(A)

Next let's convert a nested list from a previous example into a 2D array:

In [ ]:
Cdegrees = range(0, 101, 10)
Fdegrees = [9./5*C + 32 for C in Cdegrees]
table = [[C, F] for C, F in zip(Cdegrees, Fdegrees)]
print(table)

In [ ]:
# Convert this into a NumPy array:
table2 = np.array(table)
print(table2)

To see the number of elements in each dimension:

In [ ]:
print(table2.shape)

*i.e.* 11 rows and 2 columns.

Let's write a loop over all array elements of A:

In [ ]:
for i in range(table2.shape[0]):
    for j in range(table2.shape[1]):
        print('table2[%d,%d] = %g' % (i, j, table2[i,j]))

Alternatively:

In [ ]:
for index_tuple, value in np.ndenumerate(table2):
    print('index %s has value %g' % (index_tuple, table2[index_tuple]))

We can also extract slices from multi-dimensional arrays as before. For example, extract the second column:

In [ ]:
print(table2[:, 1]) # 2nd column (index 1)

Play with this more complicated example:

In [ ]:
t = np.linspace(1, 30, 30).reshape(5, 6)
print(t)

In [ ]:
print(t[1:-1:2, 2:])

## Exercise 4.3: Implement matrix-vector multiplication
A matrix $\mathbf{A}$ and a vector $\mathbf{b}$, represented in Python as a 2D array and a 1D array respectively, are given by:

$$
\mathbf{A} = \left\lbrack\begin{array}{ccc}
0 & 12 & -1\cr
-1 & -1 & -1\cr
11 & 5 & 5
\end{array}\right\rbrack
$$

$$
\mathbf{b} = \left\lbrack\begin{array}{c}
-2\cr
1\cr
7
\end{array}\right\rbrack
$$

Multiplying a matrix by a vector results in another vector $\mathbf{c}$, whose components are defined by the general rule

$$\mathbf{c}_i = \sum_j\mathbf{A}_{i,j}\mathbf{b}_j$$

* Define $\mathbf{A}$ and $\mathbf{b}$ as NumPy arrays
* Write a function called `multiply` that takes two arguments, a matrix and a vector in the form of NumPy arrays, and returns a NumPy array containing their product.
* Call this function on $\mathbf{A}$ and $\mathbf{b}$, and store the result in a variable $c$.

In [ ]:
ok.grade('question-4_3')

## Exercise 4.4: Vectorised function

Let $A_{33}$ be the two-dimensional array
$$
\mathbf{A_{33}} = \left\lbrack\begin{array}{ccc}
0 & 12 & -1\cr
-1 & -1 & -1\cr
11 & 5 & 5
\end{array}\right\rbrack
$$

Implement and apply the function
$$
f(x) = x^3 + xe^x + 1
$$
to each element in $A_{33}$. Then calculate the result of the array expression ${A_{33}}^3 + A_{33}e^{A_{33}} + 1$, and demonstrate that the end result of the two methods are the same.

In [ ]:
# def f_cubic(A):
#     ...

In [ ]:
ok.grade('question-4_4')

## Exercise 4.5: Implement matrix-matrix multiplication

Similarly to Exercise 4.3 (where a matrix is multiplied by a vector), the general rule for multiplying a $n \times m$ matrix $\mathbf{A}$ by a $m \times p$ matrix $\mathbf{B}$ results in a $n \times p$ matrix $\mathbf{C}$, whose components are defined by the general rule

$$\mathbf{C}_{i,j} = \sum^m_{k=1}\mathbf{A}_{i,k}\mathbf{B}_{k,j}$$

Again let $\mathbf{A}$ be the two-dimensional array

$$
\mathbf{A} = \left\lbrack\begin{array}{ccc}
0 & 12 & -1\cr
-1 & -1 & -1\cr
11 & 5 & 5
\end{array}\right\rbrack
$$

and let $\mathbf{B}$ be the two-dimensional array

$$
\mathbf{B} = \left\lbrack\begin{array}{ccc}
-2 & 1 & 7\cr
3 & 0 & 6\cr
2 & 3 & 5
\end{array}\right\rbrack
$$

Define `A` and `B` as NumPy arrays, and write a function `f_mult` which multiplies them together using the above rule to create a the NumPy array called `C`.

In [ ]:
# def f_mult(A,B):
#     if A.shape[1] != B.shape[0]:
#         raise RunTimeError("A should be have the same number of columns as B has rows")
#     C = np.zeros([A.shape[0],B.shape[1]])
#     ...

In [ ]:
ok.grade('question-4_5')

## Exercise 4.6: 2D array slicing


* Create a 1D NumPy array called `odd` with all of the odd numbers from 1 to 55
* Create a 2D NumPy array called `odd_sq` with all of the odd numbers from 1 to 55 in a matrix with 4 rows and 7 columns
* Using array slicing, create a 2D NumPy array called, `odd_bits`, that starts from the $2^{nd}$ column of `odd_sq` and selects every other column, of only the $2^{nd}$ and $3^{rd}$ rows of `odd_sq`

In [ ]:
ok.grade('question-4_6')

In [ ]:
ok.score()